In [1]:
#Installing packages
!pip install torch

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import MultiLabelBinarizer

from typing import List

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
from IPython.display import display

In [3]:
#Visualizing the data
module_data = pd.read_csv('German_cleaned_combined2.csv')
display(module_data)


,titelde,SDGs,lernergebnissedeUNDlerninhaltede
0,Webtechnologien,"4, 9","Studierende, die dieses Modul erfolgreich absc..."
1,Informationssysteme und Datenanalyse,"4, 9, 11, 17",Informationssysteme bilden die Basis fuer fast...
2,Nachrichtenuebertragung mit Praktikum,"4, 9",Das wesentliche Qualifikationsziel dieses Modu...
3,Nachrichtenuebertragung,"4, 9",Nach dem erfolgreichen Abschluss des Moduls si...
4,Projekt: Brain-Computer Interfacing,"3, 4, 9",Die Absolventinnen und Absolventen dieses Modu...
...,...,...,...
393,Machine Learning for Computer Security,"4, 9",Die Studierenden haben ein umfassendes Verstae...
394,Machine Learning and Security - Project,"4, 9",Nach Abschluss des Projekts verfuegen die Stud...
395,Machine Learning and Security - Bachelor Seminar,"4, 9",Im Rahmen des Seminars erwerben die Studierend...
396,Machine Learning and Security - Master Seminar,"4, 9",Im Rahmen des Seminars erwerben die Studierend...


In [5]:
#Preprocessing the dataset
descriptions = module_data['lernergebnissedeUNDlerninhaltede'].tolist()

module_data['SDGs'] = module_data['SDGs'].apply(lambda x: [label.strip() for label in x.split(',')])
labels = module_data['SDGs'].tolist()


display(labels)

[['4', '9'],
 ['4', '9', '11', '17'],
 ['4', '9'],
 ['4', '9'],
 ['3', '4', '9'],
 ['4', '9', '16'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9', '11'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9', '12'],
 ['4', '9', '11', '12', '13', '16'],
 ['4', '9', '11', '12', '13', '16'],
 ['4', '9', '12'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9', '12'],
 ['4', '7', '9', '12', '13'],
 ['4', '9', '11', '12'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9', '12'],
 ['4', '9'],
 ['4', '16'],
 ['4', '16'],
 ['4', '9', '11'],
 ['4', '5'],
 ['4', '5', '9', '16'],
 ['4', '9', '11', '17'],
 ['4', '9', '16'],
 ['4', '9', '16'],
 ['4', '9', '12'],
 ['4', '9'],
 ['4', '9'],
 ['4', '8', '9', '12'],
 ['4', '9'],
 ['3', '4', '9'],
 ['3', '4', '9'],
 ['4', '9'],
 ['3', '4', '9'],
 ['4', '9'],
 ['4', '9'],
 ['4', '7', '9', '13'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9'],
 ['4', '9', '11', '12'],
 ['4', '9', '11'],
 ['4', '9'],
 ['4', '9', '16'],
 ['4', '7', '9'],
 ['4', '9', '1

In [6]:
# Using MultiLabelBinarizer for multi-class classification
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

print(labels)

[[0 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [7]:
# Tokenizing the input texts
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
tokenized_descriptions = tokenizer(descriptions, padding=True, truncation=True, return_tensors='pt')

In [8]:
# Creating a custom dataset class
class SDGDataset(Dataset):
    def __init__(self, tokenized_texts, labels):
        self.tokenized_texts = tokenized_texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.tokenized_texts['input_ids'][idx],
            'attention_mask': self.tokenized_texts['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx])
        }

In [9]:
# Creatign an instances of the custom dataset class
dataset = SDGDataset(tokenized_descriptions, labels)

In [10]:
# Splitting the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(len(train_dataset))
print(len(val_dataset))

318
80


In [11]:
# Creating DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [12]:
# Loading the pre-trained BERT model for German
num_sdgs = len(mlb.classes_)
model = BertForSequenceClassification.from_pretrained('bert-base-german-cased', num_labels=num_sdgs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Defining training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 10

C:\Users\vader\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# Training loop
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

criterion = torch.nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].float().to(model.device)  # Move labels to the same device as inputs
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Validation
    model.eval()
    val_loss = 0
    for batch in val_dataloader:
        with torch.no_grad():
            inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].float().to(model.device)  # Move labels to the same device as inputs
            outputs = model(**inputs)
            val_loss += criterion(outputs.logits, labels).item()

    val_loss /= len(val_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss}')



Epoch 1/10, Validation Loss: 0.19381624991074203
Epoch 2/10, Validation Loss: 0.19674797793850302
Epoch 3/10, Validation Loss: 0.19684996847063302
Epoch 4/10, Validation Loss: 0.20145150311291218


KeyboardInterrupt: 

In [19]:
#Testing the model
from torch.nn.functional import sigmoid

text = "Das Modul behandelt weiterfuehrende Themen der Logik in der Informatik. Absolventen und Absolventinnen der Kurses haben ein gutes Verstaendnis fuer Anwendungen der Logik in der Informatik. Sie koennen souveraen mit logischen Systemen arbeiten, deren Komplexitaet und Ausdrucksstaerke analysieren.  Sie koennen die Anwendungsmoeglichkeiten der Logik in der Informatik gut einschaetzen und kennen einige wichtige Anwendungen der Logik z.B. in der Verifikation, der Algorithmik und Komplexitaetstheorie sowie zur Beschreibung formaler Sprachen.  Logik, Sprachen und Automaten * Logische Beschreibung regulaerer Sprachen  * Zusammenhang zwischen Logiken und Automaten Verifikation * Verifikationslogiken LTL, CTL und den mu-Kalkuel * Techniken zur automatischen Verifikation logischer Spezifikationen in Transitionssystemen Logik im Kontext relationaler Datenbanken * conjunctive queries * relationaler Kalkuel * effizientes Auswerten von Datenbankanfragen Aussdrucksstaerke logischer Systeme * Definierbarkeit in der Praedikatenlogik * Ehrenfeucht-Fraisse Spiele und  Lokalitaetssaetze Logik und Kopmplexitaet * Auswerten logischer Formeln in Strukturen, besonders Graphen. * Logische Beschreibungen von Komplexitaetsklassen Logik in der kuenstlichen Intelligenz * Constraint Satisfaction Probleme in der kuenstlichen Intelligenz"

#Tokenizng and passing the text to the model
inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(model.device)
with torch.no_grad():
    outputs = model(**inputs)

# Appling the sigmoid function to the output logits to get the probabilities
probs = sigmoid(outputs.logits)
threshold = 0.15
labels = (probs > threshold).int()

#For the testing and comparising sake, leaving labels names as numbers
label_names = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17"] 
label_names = [name for label, name in zip(labels[0], label_names) if label == 1]

print(f'Text: {text}')
print(f'Predicted labels: {label_names}')

Text: Die Studierenden sind befaehigt, digitale Systeme in einer Hardwarebeschreibungssprache (VHDL) zu beschreiben und deren Verhalten mit Hilfe der eingesetzten Entwicklungswerkzeuge zu simulieren. Sie sind in der Lage, synchrone und asynchrone hierarchische Designs mit Verhaltens- und Strukturbeschreibungen zu implementieren. Sie beherrschen sequentielle und nebenlaeufige Sprachkonstrukte. Sie kennen den Aufbau und die Funktionsweise einer Testbench und sind in der Lage, diese eigenstaendig zu entwickeln. Sie haben ausserdem Erfahrung mit dem systematischen Debugging von komplexen Schaltungen gesammelt. - Beschreibung digitaler Systeme mit einer Hardwarebeschreibungssprache (VHDL) - Wesentliche Sprachkonstrukte in VHDL (hierarchisches Design mit Verhaltens- und Strukturbeschreibungen, synchrone und asynchrone sowie sequentielle und nebenlaeufige Beschreibungen) - Beschreibung einzelner Komponenten eines RISC-Prozessors - Simulation digitaler Schaltungen (Umgang mit Entwicklungswerkz